In [82]:
import yfinance as yf
import numpy as np
import pandas as pd

tickers = {}
ticker_names = ["BTC-EUR", "SOL-EUR", "ETH-EUR", "GC=F", "SI=F"]
for ticker in ticker_names:
    tickers[ticker] = yf.download(ticker, start='2019-12-01', end='2024-11-08')
    tickers[ticker].reset_index(inplace=True)
    relativeChange = tickers[ticker].Close.pct_change(30)*100
    tickers[ticker]['RelativeChange'] = relativeChange
    tickers[ticker] = tickers[ticker].dropna() 
    tickers[ticker]['Legend_label'] = ticker
    tickers[ticker] = tickers[ticker].drop(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], axis=1)


print(tickers['BTC-EUR'])

[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_36598/82011384.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  tickers[ticker] = tickers[ticker].drop(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], axis=1)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_36598/82011384.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  tickers[ticker] = tickers[ticker].drop(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], axis=1)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_36598/82011384.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  tickers[ticker] = tickers[ticker].drop(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], axis=1)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_36598/82011384.py:

Price                       Date RelativeChange Legend_label
Ticker                                                      
30     2019-12-31 00:00:00+00:00      -4.844218      BTC-EUR
31     2020-01-01 00:00:00+00:00      -2.922370      BTC-EUR
32     2020-01-02 00:00:00+00:00      -5.327974      BTC-EUR
33     2020-01-03 00:00:00+00:00       0.584281      BTC-EUR
34     2020-01-04 00:00:00+00:00      -0.991865      BTC-EUR
...                          ...            ...          ...
1799   2024-11-03 00:00:00+00:00      15.722417      BTC-EUR
1800   2024-11-04 00:00:00+00:00      11.848685      BTC-EUR
1801   2024-11-05 00:00:00+00:00      10.301851      BTC-EUR
1802   2024-11-06 00:00:00+00:00      10.781653      BTC-EUR
1803   2024-11-07 00:00:00+00:00      24.288051      BTC-EUR

[1774 rows x 3 columns]



/tmp/ipykernel_36598/82011384.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  tickers[ticker] = tickers[ticker].drop(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], axis=1)


In [83]:
#Generate color palette from magma palette, excluding dark colors
from bokeh.palettes import Magma256
color_codes = [int(num) for num in np.linspace(100,255, len(ticker_names))]
colors = {k: v for k, v in zip(ticker_names, [Magma256[color] for color in color_codes])}
colors

{'BTC-EUR': '#892881',
 'SOL-EUR': '#C63C73',
 'ETH-EUR': '#F56C5B',
 'GC=F': '#FEB57C',
 'SI=F': '#FBFCBF'}

In [ ]:
import numpy as np
from bokeh.core.properties import field
from bokeh.plotting import figure, output_file, show, column, row
from bokeh.io import curdoc
from bokeh.models import HoverTool, ColumnDataSource, RangeTool, VSpan, LabelSet, CustomJS, CheckboxGroup

#Set dark theme to fit with magma palette
curdoc().theme='dark_minimal'
curdoc().title = "Interactive crypto and precious metal stock price viewer"
#The file to save the model
output_file("test.html")

#Instantiate figure object
dates = np.array(tickers['BTC-EUR']['Date'], dtype=np.datetime64)
graph = figure(height=500, width=1000, x_axis_type = "datetime", title = "Relative price changes", 
               tools='tap,xpan,reset', x_axis_location = "above", x_range =(dates[-365], dates[-1])) 

select = figure(title="Drag the middle and edges of the selection box to change the range",
                height=150, width=1000, y_range=graph.y_range, x_axis_type="datetime",
                tools="", toolbar_location=None)
range_tool = RangeTool(x_range=graph.x_range)
range_tool.overlay.fill_color = "#900C3F"
range_tool.overlay.fill_alpha = 0.2


#Names axes
graph.xaxis.axis_label = 'Date'
graph.yaxis.axis_label = '30-day price change (%)'

#Plotting the line graph

renderers = []
select_renderers = []

#Adding prices
for ticker_name in ticker_names:

    source = ColumnDataSource(tickers[ticker_name])
    color = colors[ticker_name]
    l = graph.line('Date_', 'RelativeChange_', source=source, color=color, legend_label = ticker_name)
    renderers.append(l)

    select_renderers.append(select.line('Date_', 'RelativeChange_', source=source, color=color))

#Adding vspans for historical events
date_strings = ["2019-12-31", "2020-03-11", "2021-02-08", "2021-09-14", "2022-02-24", "2023-03-10", "2023-10-07", "2024-11-05"]
datetime_array = np.array(date_strings, dtype=np.datetime64)
label_strings = ["COVID-19 identified", "COVID-19 declared pandemic", "Tesla invests $1.5B in BTC", "SOL worldwide outage", "Russia-Ukraine War begins", "Silicon Valley Bank Collapse", "Israel-Hamas conflicts begin", "US Election Day"]

vspan_df = pd.DataFrame(dict(dates=datetime_array, width=np.full(len(datetime_array),2), labels=label_strings))
vspan_source = ColumnDataSource(vspan_df)

vspan = VSpan(x=field("dates"), line_width=field("width"), line_color="white", line_dash="dashed")

myMax = -10000
for ticker_name in tickers:
    currentMax = tickers[ticker_name]['RelativeChange'].max()
    if myMax < currentMax:
        myMax = currentMax


labels = LabelSet(x="dates", y=myMax*0.8, text="labels", x_offset=5, source=vspan_source, text_color="white")
graph.add_glyph(vspan_source, vspan)
select.add_glyph(vspan_source,vspan)
graph.add_layout(labels)

#Adding CheckboxGroup for toggling
checkbox_group = CheckboxGroup(labels=ticker_names, active = list(range(len(ticker_names))))

checkbox_callback = CustomJS(args=dict(renderers = renderers, select_renderers = select_renderers, checkbox_group = checkbox_group, tickers=tickers, graph=graph, select=select, labels=labels), code ="""
                             console.log(tickers)
                             console.log("Beginning is:"+graph.x_range.start)
                             console.log("End is:" +graph.x_range.end)                                
                             console.log(labels)  
                             let myMax = -10000

                             for (let i = 0; i < checkbox_group.labels.length; i++){
                                var contains = checkbox_group.active.includes(i)
                                renderers[i].visible = contains;
                                select_renderers[i].visible = contains
                                //graph.legend.items[i].visible = contains

                                if (contains){
                                    let [_, value] = Object.entries(tickers)[i]
                                    let relativeChange = value.filter((_, index) => index % 3 === 1)
                                    let currentMax = Math.max(...relativeChange)
                                    if (myMax < currentMax){
                                        myMax = currentMax
                                    }
                                }
                             
                             }
                             console.log("myMax is: "+myMax)
                             graph.y_range.end = myMax * 1.1
                             //select.y_range.end = myMax * 1.1
                             console.log("Y-Range is:" +graph.y_range.end)
                             //labels.y = myMax * 1.1 * 0.8
                    """)

checkbox_group.js_on_change("active", checkbox_callback)

#Adding callback to move labels when y axis changes
label_position_callback = CustomJS(args=dict(labels=labels, graph=graph), code="""
    var y_end = graph.y_range.end
    var new_y = 0.8 * y_end
    labels.y = new_y       
""")
graph.y_range.js_on_change("end", label_position_callback)

#Adding callback to rescale y-axis to fit current timeframe
y_axis_rescale_callback = CustomJS(args=dict(checkbox_group = checkbox_group, tickers=tickers, graph=graph), code="""
        let myMax = -10000
        let start = graph.x_range.start
        let end = graph.x_range.end
                                   
        for (let i = 0; i < checkbox_group.labels.length; i++){
            var contains = checkbox_group.active.includes(i)

            if (contains){
                let [_, value] = Object.entries(tickers)[i]
                let extractedValues = []
                
                for (let j = 0; j < value.length; j+=3){
                    if (value[j] >= start && value[j] <= end ){
                        extractedValues.push(value[j+1])               
                    }                   
                }

                let currentMax = Math.max(...extractedValues)
                if (myMax < currentMax){
                    myMax = currentMax
                }
            }
    
        }
        console.log("myMax is: " +myMax)
        graph.y_range.end = myMax * 1.1                            
""")
graph.x_range.js_on_change("end", y_axis_rescale_callback)



#Adding miscellaneous stylization
graph.legend.location = 'top_left'

hover = HoverTool(tooltips=[("Crypto name", "@Legend_label_"),("Date", "@Date_{%F}"), ("Price change (%)", "@RelativeChange_{0.00}")],
                   renderers=renderers, formatters={'@Date_': 'datetime'})
graph.add_tools(hover)
select.ygrid.grid_line_color=None
select.add_tools(range_tool)

show(column(row(graph, checkbox_group),select))

/tmp/ipykernel_36598/3795429081.py:14: UserWarning: no explicit representation of timezones available for np.datetime64
  dates = np.array(tickers['BTC-EUR']['Date'], dtype=np.datetime64)
